<a href="https://colab.research.google.com/github/victormurcia/CTS_Test/blob/main/CTS_Parser_Development_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up the Environment

In [1]:
#I need to import locale to ensure that the encoding is set to UTF-8 (weird Google Colab bug)
import locale
locale.getpreferredencoding = lambda: "UTF-8"

#Check the current build in Google Colab
!cat /etc/*release
print('\n')

#Check CUDA version
!nvcc --version
print('\n')

#Ensure that the required packages are installed in the current environment
!pip install numpy --quiet
!pip install pandas --quiet
!pip install spacy==3.4.4 --quiet
!pip install scispacy --quiet
!pip install medspacy --quiet
!pip install scispacy --quiet
!pip install negspacy --quiet
!pip install seaborn --quiet
!pip install matplotlib --quiet
!pip install "dask[complete]" --quiet
!pip install ipywidgets --quiet
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet 
print('\n')

#Spacy models used for processing biomedical, scientific, or clinical text 
#Spacy pipeline for biomedical data.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz --quiet
#Spacy pipeline for biomedical data. Has a larger vocabulary and 50k word vectors
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz --quiet
#This one is another spacy pipeline with 785k vocabulary and uses scibert-base as a transformer model
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_scibert-0.5.1.tar.gz --quiet
#Spacy pipeline for biomedical data with 600k word vectors
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_lg-0.5.1.tar.gz --quiet
#A spaCy NER model trained on the CRAFT corpus.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_craft_md-0.5.1.tar.gz --quiet
#A spaCy NER model trained on the JNLPBA corpus.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_jnlpba_md-0.5.1.tar.gz --quiet
#A spaCy NER model trained on the BC5CDR corpus.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz --quiet
#A spaCy NER model trained on the BIONLP13CG corpus.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bionlp13cg_md-0.5.1.tar.gz --quiet
#This is the med7 transformer model found here: https://github.com/kormilitzin/med7
!pip install https://huggingface.co/kormilitzin/en_core_med7_trf/resolve/main/en_core_med7_trf-any-py3-none-any.whl --quiet
#This is the med7 vector model 
!pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl --quiet
print('\n')

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION="Ubuntu 20.04.5 LTS"
NAME="Ubuntu"
VERSION="20.04.5 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.5 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 20

In [2]:
#Import the required libraries/packages
#General utilities
import numpy as np
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import os, random, time,sys, re
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from tqdm import tqdm

#NLP Stuff
#Spacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS #Load stopwords
from spacy.language import Language
from spacy.tokenizer import Tokenizer
#Scispacy
import scispacy
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector
from scispacy.hyponym_detector import HyponymDetector
#Medspacy
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent
from negspacy.negation import Negex
#NLTK

#Enable data to be extracted from my Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the Data

In [3]:
#Load the Synthetic Veteran Suicide Dataset from my Google Drive
data_dir = r'/content/drive/MyDrive/csv_usa_100k/' #Establish location of data
print('The data is located at:')
print(data_dir, '\n')

def find_csv_files(data_dir):
  """
  This function finds the .csv files located in data_dir and returns an alphabetically sorted file list.

  Parameters:
  data_dir (str) : Directory of files
  
  Returns:
  data_list (list) : list of .csv files
  """
  data_list = [f for f in os.listdir(data_dir) if f.endswith('.csv')] #List the .csv files
  print('The data files are:')
  data_list.sort() #Sort the .csv files alphabetically
  print(data_list,'\n')

  #Get names of .csv files without the extension for naming stuff later
  f_names = [s.replace(".csv","") for s in data_list]

  return data_list,f_names

#Get .csv files
data_list,f_names = find_csv_files(data_dir)

The data is located at:
/content/drive/MyDrive/csv_usa_100k/ 

The data files are:
['allergies.csv', 'careplans.csv', 'conditions.csv', 'devices.csv', 'encounters.csv', 'imaging_studies.csv', 'immunizations.csv', 'medications.csv', 'observations.csv', 'organizations.csv', 'patients.csv', 'payer_transitions.csv', 'payers.csv', 'procedures.csv', 'providers.csv', 'supplies.csv'] 



Some of these files have information that won't be useful for me so I'll remove them from my list now.  The files that I'll be removing are: encounters, organizations, payer_transitions, payers, and providers. 

In [4]:
files2remove = ['encounters.csv', 'organizations.csv', 'payer_transitions.csv', 'payers.csv', 'providers.csv'] 
def remove_files(data_list,files2remove):
  for f in data_list:
    if f in files2remove:
      data_list.remove(f)

  res = filter(lambda i: i not in files2remove, data_list)
  return list(res)

data_list = remove_files(data_list,files2remove)
print(data_list)

['allergies.csv', 'careplans.csv', 'conditions.csv', 'devices.csv', 'imaging_studies.csv', 'immunizations.csv', 'medications.csv', 'observations.csv', 'patients.csv', 'procedures.csv', 'supplies.csv']


In [5]:
#Add data path to beginning of each element of list so that I can easily access them later
def prepend(data_list, str):
    str += '% s'
    new_list = [str % i for i in data_list]
    return new_list

#Add 
csv_list = prepend(data_list,data_dir)
print('Full path of .csv files \n')
csv_list

Full path of .csv files 



['/content/drive/MyDrive/csv_usa_100k/allergies.csv',
 '/content/drive/MyDrive/csv_usa_100k/careplans.csv',
 '/content/drive/MyDrive/csv_usa_100k/conditions.csv',
 '/content/drive/MyDrive/csv_usa_100k/devices.csv',
 '/content/drive/MyDrive/csv_usa_100k/imaging_studies.csv',
 '/content/drive/MyDrive/csv_usa_100k/immunizations.csv',
 '/content/drive/MyDrive/csv_usa_100k/medications.csv',
 '/content/drive/MyDrive/csv_usa_100k/observations.csv',
 '/content/drive/MyDrive/csv_usa_100k/patients.csv',
 '/content/drive/MyDrive/csv_usa_100k/procedures.csv',
 '/content/drive/MyDrive/csv_usa_100k/supplies.csv']

## Create Dataframes With Dask

In [79]:
cols = list(pd.read_csv(csv_list[8], nrows =0)) #Only read the column headers
print(cols)

['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']


In [6]:
#After checking the contents of all the .csv files, I decided that the following columns across all .csv files will not be 
#included during loading
omit_cols = ['START','STOP','ENCOUNTER','BASE_COST','PAYER_COVERAGE','DATE','TYPE','DEATHDATE', 
             'SSN', 'DRIVERS', 'PASSPORT', 'MAIDEN','ADDRESS','FIRST', 'LAST', 'SUFFIX','START_YEAR', 
             'END_YEAR', 'PAYER','BASE_COST','QUANTITY']

In [7]:
def read_all_csvs(csv_list,omit_cols):
  """
  This function reads all the csv files in a list, loads each of them into a dask dataframe, and 
  then places them into a list called df_list that can be accessed later.

  Parameters:
  csv_list (list)  : List containing csv files
  omit_cols (list) : List containing names of columns to be omitted during dataframe loading

  Returns:
  df_list (list) :list containing dask dataframes
  """
  #Start timer
  start_time = time.time()

  #Initialize array that will contain the loaded dataframes
  df_list = []

  #Number of .csv files to load
  nFiles = len(csv_list)

  #Iterate through the csv files and load the dataframes
  for j in range(nFiles):
    cols    = list(pd.read_csv(csv_list[j], nrows = 0)) #Only read header row for column names
    temp_df = dd.read_csv(csv_list[j] , usecols =[i for i in cols if i not in omit_cols],assume_missing=True,dtype={'VALUE': 'object'}) #Make dask df
    df_list.append(temp_df)
  
  #End timer
  print("Loading of " + str(nFiles) + " .csv files took: %.2f seconds" % (time.time() - start_time))

  return df_list

df_list = read_all_csvs(csv_list,omit_cols)

Loading of 11 .csv files took: 9.95 seconds


## EDA

In [82]:
#Visualize descriptions columns of dataframe 
def make_countplots(df,name,n):

  if n < 25:
    val = n
  else:
    val = 25
  b = sns.countplot(data=df,y='DESCRIPTION',order = df['DESCRIPTION'].value_counts().index[:25])
  b.axes.set_title("Top "+ str(val) + ' ' + name.capitalize() + " Present in Dataset",fontsize=25)
  b.set_xlabel("Counts",fontsize=15)
  b.set_ylabel(name.capitalize(),fontsize=15)
  b.tick_params(labelsize=8)
  plt.show()

#View the number of unique patients in each dataframe
@interact
def show_nunique_vals_in_df(x=widgets.IntSlider(min=0,max=len(df_list)-1,step=1,value=0)):
  nPatients = df_list[x]['PATIENT'].nunique().compute()
  nDescriptions = df_list[x]['DESCRIPTION'].nunique().compute()
  print('The ' + f_names[x] + '.csv file has' ,nPatients, 'unique patients and',nDescriptions,'unique',f_names[x])
  make_countplots(df_list[x].compute(),f_names[x],nDescriptions)

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

## Restructuring and Merging Dataframes

In [8]:
def restructure_dfs(ref_df,source_df,nPatients):
    """
    This function combines the observations for a patient in the Synthetic Veteran Suicide Dataset into a 
    single row in a new dataframe. Doing this should help with the NLP processing later.
    
    Parameters
    ----------
    nPatients : int,  How many patients to process?
    ref_df    : df,   Dataframe containing restructured patient data (should be the first dataframe that was 
                      restructured)
    source_df : df,   Dataframe that contains information to be restructured
    ----------
    """
    # Create empty patient dataframe
    columns = source_df.columns
    temp_df = pd.DataFrame(index=np.arange(nPatients),columns=columns)    
    
    for i in range(ref_df.shape[0]):
        patient = ref_df['PATIENT'][i] #Patient ID    
        #dummy_df   = source_df[source_df['PATIENT'] == patient]
        
        #Programatically populate the dataframe by first making lists out of the columns and then inserting
        #these lists into cells in the resulting dataframe
        for k,col in enumerate(columns):
            if col == 'PATIENT':
                temp_df.at[i, 'PATIENT'] = patient    
            else:
                temp_df.at[i, col] = source_df[col].tolist()
                
    return temp_df

def make_patient_df(nPatients,df,df_list, s_list, init_rand=False):
    """
    This function combines the observations for a patient in the Synthetic Veteran Suicide Dataset into a 
    single row in a new dataframe. Doing this should help with the NLP processing later.
    
    Parameters
    ----------
    nPatients : int,  How many patients to process?
    df        : df,   Dataframe containing patient data
    df_list   : list, list of all dataframes in dataset
    init_rand : bool, Use random values to choose patients? Defaults to False so only first 10 patients are selected
    s_list    : list, List of strings to append to column names in the reformatted dataframes 
    ----------
    """

    #Start timer
    #start_time = time.time()

    #Determine number of unique patients in dataframe
    unique_patients = df['PATIENT'].nunique()
    
    #Initialize iterators and patient index
    i = 0; j = random.randrange(unique_patients)
    
    #Initialize index array
    index_list = []
    
    #Make list of restructured dataframes
    rs_df_list = []
    
    for i in range(nPatients):
      # Create empty patient dataframe
      columns = df.columns
      new_df  = pd.DataFrame(index=np.arange(nPatients),columns=columns)
      
      #Select a random patient from the existing dataframe (random sample)
      if init_rand == True:
        j = random.randrange(unique_patients)                
      else:
        j = i 
      
      #Check that patient id has not already been added
      if j not in index_list:
        index_list.append(j)
      else:
        while j in index_list:
            j = random.randrange(unique_patients) 

        index_list.append(j)
      
      #Subset the dataframe so as to only show the results associated with a given patient
      patient    = df['PATIENT'][j] #Patient ID
      dummy_df   = df[df['PATIENT'] == patient]
      
      #Programatically populate the dataframe by first making lists out of the columns and then inserting
      #these lists into cells in the resulting dataframe
      for k,col in enumerate(columns):
        if col == 'PATIENT':
            new_df.at[i, 'PATIENT'] = patient    
        else:
            new_df.at[i, col] = dummy_df[col].tolist()
      
      #Append the restructured dataframe to the list of dfs
      rs_df_list.append(new_df)

      #1. Select the source dataframe to be converted from df_list
      #2. Subset this dataframe so that only the rows with the current patient ids are selected
      for m,cdf in enumerate(df_list[1:]):
        columns = cdf.columns

        # Create empty patient dataframe
        new_df  = pd.DataFrame(index=np.arange(nPatients),columns=columns)

        if 'PATIENT' not in columns:
          dummy_df = cdf[cdf['Id'] == patient]
        else:
          dummy_df = cdf[cdf['PATIENT'] == patient]

        dummy_df = dummy_df.compute()

        for k,col in enumerate(columns):
          if 'PATIENT' not in columns:
            if col == 'Id':
              new_df.at[i, 'PATIENT'] = patient    
            else:
              new_df.at[i, col] = dummy_df[col].tolist()
          else:
            if col == 'PATIENT':
              new_df.at[i, 'PATIENT'] = patient    
            else:
              new_df.at[i, col] = dummy_df[col].tolist()

        rs_df_list.append(new_df)

    #Rename columns so as to have a suffix included for easy traceback to source file
    for m,df in enumerate(rs_df_list):
      columns = df.columns
      if 'PATIENT' not in columns:
        df.columns = [x + s_list[m] if x != 'Id' else x for x in columns]
      else:
        df.columns = [x + s_list[m] if x != 'PATIENT' else x for x in columns]    

    #print('These are the indices for the patient IDs used to generate the dataframe below. \n',index_list)

    #End timer
    #print("Reformatting dataframes took: %.2f seconds" % (time.time() - start_time))

    return rs_df_list  

s_list = ['_als','_cps','_cds','_dvs','_iss','_ims','_mds','_obs','_pts','_prs','_sps']
#rs_df_list  = make_patient_df(1, df_list[0].compute(),df_list, s_list, init_rand=True)

In [40]:
def patient_df_wrapper(nPatients):
  """
  This function simply iterates over the make_patient_df function so as to generate the restructured dataframes for each patient. It concatenates the resulting dataframes into a single dataframe that will be
  use for subsequent processing/analysis. 
  """

  print('Starting process...')

  #Start timer
  start_time = time.time()

  #Initialize array to contain dataframes
  multiple_patients_df_list = []

  #Generate the patient dataframes
  for i in tqdm(range(nPatients)):
    rs_df_list  = make_patient_df(1, df_list[0].compute(),df_list, s_list, init_rand=True)
    dfs = [df.set_index('PATIENT') for df in rs_df_list] #Set the index for each dataframe to be the PATIENT Id
    df = pd.concat(dfs, axis=1) #
    multiple_patients_df_list.append(df)
  
  df = pd.concat(multiple_patients_df_list, axis=0)

  #End timer
  print("\n Multipatient dataframe generated after: %.2f seconds" % (time.time() - start_time))

  return df

#Generate dataframe for 10 patients
multi_patient_df = patient_df_wrapper(10)

#Save resulting dataframe to a .csv file
multi_patient_df.to_csv('multi_veteran_df.csv')

#Visualize dataframe
multi_patient_df

Starting process...


100%|██████████| 10/10 [57:07<00:00, 342.78s/it]


 Multipatient dataframe generated after: 3427.84 seconds


,CODE_als,DESCRIPTION_als,Id_cps,CODE_cps,DESCRIPTION_cps,REASONCODE_cps,REASONDESCRIPTION_cps,CODE_cds,DESCRIPTION_cds,CODE_dvs,...,LAT_pts,LON_pts,HEALTHCARE_EXPENSES_pts,HEALTHCARE_COVERAGE_pts,CODE_prs,DESCRIPTION_prs,REASONCODE_prs,REASONDESCRIPTION_prs,CODE_sps,DESCRIPTION_sps
PATIENT,,,,,,,,,,,,,,,,,,,,,
10546cba-e069-4733-8355-d0da19937a88,"[424213003.0, 419474003.0, 232350006.0, 232347...","[Allergy to bee venom, Allergy to mould, House...","[2e9f356f-9b6b-4dc5-a51b-b0e66c9cbed7, 1d614a6...","[53950000.0, 384758001.0, 53950000.0, 69972800...","[Respiratory therapy, Self-care interventions ...","[10509002.0, nan, 10509002.0, 233678006.0, 157...","[Acute bronchitis (disorder), nan, Acute bronc...","[10509002.0, 65363002.0, 10509002.0, 233678006...","[Acute bronchitis (disorder), Otitis media, Ac...",[],...,[34.54533234020288],[-92.40716788469534],[1070643.95],[7055.449999999999],"[269911007.0, 430193006.0, 395142003.0, 430193...","[Sputum examination (procedure), Medication Re...","[10509002.0, nan, nan, nan, nan, nan, 23367800...","[Acute bronchitis (disorder), nan, nan, nan, n...","[409534002.0, 713779008.0, 469673003.0, 706724...",[Disposable air-purifying respirator (physical...
89bbecbb-bce6-40a9-a6c8-43bad8ea71a7,"[424213003.0, 419474003.0, 232350006.0, 232347...","[Allergy to bee venom, Allergy to mould, House...","[77e30ee7-423f-40a7-8d20-49ad038a60fb, e707e3a...","[384758001.0, 53950000.0, 225358003.0, 3856910...","[Self-care interventions (procedure), Respirat...","[nan, 10509002.0, 284549007.0, 16114001.0, 105...","[nan, Acute bronchitis (disorder), Laceration ...","[65363002.0, 241929008.0, 10509002.0, 28454900...","[Otitis media, Acute allergic reaction, Acute ...",[],...,[32.20089854172043],[-110.91995776771374],[1222393.64],[2541.9],"[430193006.0, 430193006.0, 430193006.0, 313191...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, 10509002.0, nan, nan, 284...","[nan, nan, nan, nan, Acute bronchitis (disorde...",[],[]
df91f29a-1717-4794-a358-21dc4d395f75,"[419474003.0, 232350006.0, 232347008.0, 418689...","[Allergy to mould, House dust mite allergy, Da...","[d0997e10-28a0-4a49-9714-841cb8e9c03c, 05e3415...","[384758001.0, 385691007.0, 385691007.0, 711282...","[Self-care interventions (procedure), Fracture...","[nan, 58150001.0, 16114001.0, 24079001.0, 2336...","[nan, Fracture of clavicle, Fracture of ankle,...","[65363002.0, 58150001.0, 65363002.0, 43878008....","[Otitis media, Fracture of clavicle, Otitis me...",[],...,[39.64483039653518],[-121.84955887338783],[73660.2],[12890.48],"[430193006.0, 430193006.0, 395142003.0, 168594...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, 58150001.0, 43878008.0, n...","[nan, nan, nan, nan, Fracture of clavicle, Str...","[409534002.0, 713779008.0, 469673003.0, 706724...",[Disposable air-purifying respirator (physical...
7df665af-7442-43f1-9c54-714937558930,"[419474003.0, 232350006.0, 232347008.0, 418689...","[Allergy to mould, House dust mite allergy, Da...","[b47bb4b9-5580-49c5-960e-ba00f8ea6b73, 440af73...","[384758001.0, 386522008.0, 408869004.0, 912510...","[Self-care interventions (procedure), Overacti...","[nan, 192127007.0, 39848009.0, 44465007.0, 475...","[nan, Child attention deficit disorder, Whipla...","[446096008.0, 444814009.0, 192127007.0, 195662...","[Perennial allergic rhinitis, Viral sinusitis ...",[],...,[36.54540251362364],[-121.80952824776747],[28040.119999999995],[0.0],"[430193006.0, 430193006.0, 430193006.0, 430193...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[]
3d878978-037c-4999-b9d5-3a85f1102caf,"[300916003.0, 419474003.0, 232347008.0, 418689...","[Latex allergy, Allergy to mould, Dander (anim...","[d78eb586-f70b-4ba6-b6e9-b56818ed890f, 500b676...","[385691007.0, 384758001.0, 91251008.0, 3865220...","[Fracture care, Self-care interventions (proce...","[33737001.0, nan, 44465

In [41]:
#I don't need the Id columns so I'll drop them
df2 = multi_patient_df[multi_patient_df.columns.drop(list(multi_patient_df.filter(regex='Id')))]
df2

,CODE_als,DESCRIPTION_als,CODE_cps,DESCRIPTION_cps,REASONCODE_cps,REASONDESCRIPTION_cps,CODE_cds,DESCRIPTION_cds,CODE_dvs,DESCRIPTION_dvs,...,LAT_pts,LON_pts,HEALTHCARE_EXPENSES_pts,HEALTHCARE_COVERAGE_pts,CODE_prs,DESCRIPTION_prs,REASONCODE_prs,REASONDESCRIPTION_prs,CODE_sps,DESCRIPTION_sps
PATIENT,,,,,,,,,,,,,,,,,,,,,
10546cba-e069-4733-8355-d0da19937a88,"[424213003.0, 419474003.0, 232350006.0, 232347...","[Allergy to bee venom, Allergy to mould, House...","[53950000.0, 384758001.0, 53950000.0, 69972800...","[Respiratory therapy, Self-care interventions ...","[10509002.0, nan, 10509002.0, 233678006.0, 157...","[Acute bronchitis (disorder), nan, Acute bronc...","[10509002.0, 65363002.0, 10509002.0, 233678006...","[Acute bronchitis (disorder), Otitis media, Ac...",[],[],...,[34.54533234020288],[-92.40716788469534],[1070643.95],[7055.449999999999],"[269911007.0, 430193006.0, 395142003.0, 430193...","[Sputum examination (procedure), Medication Re...","[10509002.0, nan, nan, nan, nan, nan, 23367800...","[Acute bronchitis (disorder), nan, nan, nan, n...","[409534002.0, 713779008.0, 469673003.0, 706724...",[Disposable air-purifying respirator (physical...
89bbecbb-bce6-40a9-a6c8-43bad8ea71a7,"[424213003.0, 419474003.0, 232350006.0, 232347...","[Allergy to bee venom, Allergy to mould, House...","[384758001.0, 53950000.0, 225358003.0, 3856910...","[Self-care interventions (procedure), Respirat...","[nan, 10509002.0, 284549007.0, 16114001.0, 105...","[nan, Acute bronchitis (disorder), Laceration ...","[65363002.0, 241929008.0, 10509002.0, 28454900...","[Otitis media, Acute allergic reaction, Acute ...",[],[],...,[32.20089854172043],[-110.91995776771374],[1222393.64],[2541.9],"[430193006.0, 430193006.0, 430193006.0, 313191...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, 10509002.0, nan, nan, 284...","[nan, nan, nan, nan, Acute bronchitis (disorde...",[],[]
df91f29a-1717-4794-a358-21dc4d395f75,"[419474003.0, 232350006.0, 232347008.0, 418689...","[Allergy to mould, House dust mite allergy, Da...","[384758001.0, 385691007.0, 385691007.0, 711282...","[Self-care interventions (procedure), Fracture...","[nan, 58150001.0, 16114001.0, 24079001.0, 2336...","[nan, Fracture of clavicle, Fracture of ankle,...","[65363002.0, 58150001.0, 65363002.0, 43878008....","[Otitis media, Fracture of clavicle, Otitis me...",[],[],...,[39.64483039653518],[-121.84955887338783],[73660.2],[12890.48],"[430193006.0, 430193006.0, 395142003.0, 168594...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, 58150001.0, 43878008.0, n...","[nan, nan, nan, nan, Fracture of clavicle, Str...","[409534002.0, 713779008.0, 469673003.0, 706724...",[Disposable air-purifying respirator (physical...
7df665af-7442-43f1-9c54-714937558930,"[419474003.0, 232350006.0, 232347008.0, 418689...","[Allergy to mould, House dust mite allergy, Da...","[384758001.0, 386522008.0, 408869004.0, 912510...","[Self-care interventions (procedure), Overacti...","[nan, 192127007.0, 39848009.0, 44465007.0, 475...","[nan, Child attention deficit disorder, Whipla...","[446096008.0, 444814009.0, 192127007.0, 195662...","[Perennial allergic rhinitis, Viral sinusitis ...",[],[],...,[36.54540251362364],[-121.80952824776747],[28040.119999999995],[0.0],"[430193006.0, 430193006.0, 430193006.0, 430193...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[]
3d878978-037c-4999-b9d5-3a85f1102caf,"[300916003.0, 419474003.0, 232347008.0, 418689...","[Latex allergy, Allergy to mould, Dander (anim...","[385691007.0, 384758001.0, 91251008.0, 3865220...","[Fracture care, Self-care interventions (proce...","[33737001.0, nan, 44465007.0, 192127007.0, 596...","[Fracture of rib, nan, Sprain of ankle, Child ...","[33737001.0, 65363002.0, 241929008.0, 65363002...","[Fracture of rib, Otitis media, Acute allergic...",[],[],...,[39.653384345431405],[-104.83010021193084],[9

In [42]:
#Rename columns to something more intuitive
new_names = ['AllergyCode',
 'Allergy',
 'CareplanCode',
 'Careplan',
 'CareplanReason',
 'CareplanReasonDescription',
 'ConditionCode',
 'Condition',
 'DeviceCode',
 'Device',
 'DeviceUDI',
 'ImageBodysiteCode',
 'ImageBodysiteDescription',
 'ImageModalityCode',
 'ImageModalityDescription',
 'ImageSOPCode',
 'ImageSOPDescription',
 'ImmunizationCode',
 'Immunization',
 'MedicationCode',
 'Medication',
 'MedicationDispenses',
 'MedicationCost',
 'MedicationReason',
 'MedicationReasonDescription',
 'ObservationCode',
 'Observation',
 'ObservationValue',
 'ObservationUnit',
 'PatientBirthday',
 'PatientPrefix',
 'PatientMarital',
 'PatientRace',
 'PatientEthnicity',
 'PatientGender',
 'PatientBirthplace',
 'PatientCity',
 'PatientState',
 'PatientCounty',
 'PatientZIP',
 'PatientLAT',
 'PatientLON',
 'PatientHEALTHCARE_EXPENSES',
 'PatientHEALTHCARE_COVERAGE',
 'ProcedureCode',
 'ProcedureDescription',
 'ProcedureReason',
 'ProcedureReasonDescription',
 'SupplyCode',
 'SupplyDescription']

df2.columns = new_names
df2

,AllergyCode,Allergy,CareplanCode,Careplan,CareplanReason,CareplanReasonDescription,ConditionCode,Condition,DeviceCode,Device,...,PatientLAT,PatientLON,PatientHEALTHCARE_EXPENSES,PatientHEALTHCARE_COVERAGE,ProcedureCode,ProcedureDescription,ProcedureReason,ProcedureReasonDescription,SupplyCode,SupplyDescription
PATIENT,,,,,,,,,,,,,,,,,,,,,
10546cba-e069-4733-8355-d0da19937a88,"[424213003.0, 419474003.0, 232350006.0, 232347...","[Allergy to bee venom, Allergy to mould, House...","[53950000.0, 384758001.0, 53950000.0, 69972800...","[Respiratory therapy, Self-care interventions ...","[10509002.0, nan, 10509002.0, 233678006.0, 157...","[Acute bronchitis (disorder), nan, Acute bronc...","[10509002.0, 65363002.0, 10509002.0, 233678006...","[Acute bronchitis (disorder), Otitis media, Ac...",[],[],...,[34.54533234020288],[-92.40716788469534],[1070643.95],[7055.449999999999],"[269911007.0, 430193006.0, 395142003.0, 430193...","[Sputum examination (procedure), Medication Re...","[10509002.0, nan, nan, nan, nan, nan, 23367800...","[Acute bronchitis (disorder), nan, nan, nan, n...","[409534002.0, 713779008.0, 469673003.0, 706724...",[Disposable air-purifying respirator (physical...
89bbecbb-bce6-40a9-a6c8-43bad8ea71a7,"[424213003.0, 419474003.0, 232350006.0, 232347...","[Allergy to bee venom, Allergy to mould, House...","[384758001.0, 53950000.0, 225358003.0, 3856910...","[Self-care interventions (procedure), Respirat...","[nan, 10509002.0, 284549007.0, 16114001.0, 105...","[nan, Acute bronchitis (disorder), Laceration ...","[65363002.0, 241929008.0, 10509002.0, 28454900...","[Otitis media, Acute allergic reaction, Acute ...",[],[],...,[32.20089854172043],[-110.91995776771374],[1222393.64],[2541.9],"[430193006.0, 430193006.0, 430193006.0, 313191...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, 10509002.0, nan, nan, 284...","[nan, nan, nan, nan, Acute bronchitis (disorde...",[],[]
df91f29a-1717-4794-a358-21dc4d395f75,"[419474003.0, 232350006.0, 232347008.0, 418689...","[Allergy to mould, House dust mite allergy, Da...","[384758001.0, 385691007.0, 385691007.0, 711282...","[Self-care interventions (procedure), Fracture...","[nan, 58150001.0, 16114001.0, 24079001.0, 2336...","[nan, Fracture of clavicle, Fracture of ankle,...","[65363002.0, 58150001.0, 65363002.0, 43878008....","[Otitis media, Fracture of clavicle, Otitis me...",[],[],...,[39.64483039653518],[-121.84955887338783],[73660.2],[12890.48],"[430193006.0, 430193006.0, 395142003.0, 168594...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, 58150001.0, 43878008.0, n...","[nan, nan, nan, nan, Fracture of clavicle, Str...","[409534002.0, 713779008.0, 469673003.0, 706724...",[Disposable air-purifying respirator (physical...
7df665af-7442-43f1-9c54-714937558930,"[419474003.0, 232350006.0, 232347008.0, 418689...","[Allergy to mould, House dust mite allergy, Da...","[384758001.0, 386522008.0, 408869004.0, 912510...","[Self-care interventions (procedure), Overacti...","[nan, 192127007.0, 39848009.0, 44465007.0, 475...","[nan, Child attention deficit disorder, Whipla...","[446096008.0, 444814009.0, 192127007.0, 195662...","[Perennial allergic rhinitis, Viral sinusitis ...",[],[],...,[36.54540251362364],[-121.80952824776747],[28040.119999999995],[0.0],"[430193006.0, 430193006.0, 430193006.0, 430193...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[]
3d878978-037c-4999-b9d5-3a85f1102caf,"[300916003.0, 419474003.0, 232347008.0, 418689...","[Latex allergy, Allergy to mould, Dander (anim...","[385691007.0, 384758001.0, 91251008.0, 3865220...","[Fracture care, Self-care interventions (proce...","[33737001.0, nan, 44465007.0, 192127007.0, 596...","[Fracture of rib, nan, Sprain of ankle, Child ...","[33737001.0, 65363002.0, 241929008.0, 65363002...","[Fracture of rib, Otitis media, Acute allergic...",[],[],...,[39.653384345431405],[-10

In [44]:
test_str = ",".join(df2['Allergy'][0])
test_str

'Allergy to bee venom,Allergy to mould,House dust mite allergy,Dander (animal) allergy,Allergy to grass pollen,Allergy to tree pollen,Allergy to eggs,Allergy to nut'

## Accessing Clinical Trials

In [12]:
#Query for trials
cond = 'allergy'#input('Enter the disease condition to find clinical trials: ')
a = 'https://clinicaltrials.gov/api/query/study_fields?expr='
b = '&fields=NCTId%2CBriefTitle%2CCondition%2COverallStatus%2CLeadSponsorName%2CEligibilityCriteria'
c = '&min_rnk=1&max_rnk=1000&fmt=csv'
q=(a + cond + b + c)
print(q)
qtrials = pd.read_csv(q,skiprows=10)

@interact
def show_recruiting_studies(column=['OverallStatus','Condition'], 
                            x = ['Recruiting','Completed','Unknown status'],
                            y = ['Food','Antibiotic']
                            ):
    if column == 'OverallStatus':
      return qtrials.loc[qtrials[column] == x]
    elif column == 'Condition':
      return qtrials[qtrials['Condition'].str.contains(y)]

https://clinicaltrials.gov/api/query/study_fields?expr=allergy&fields=NCTId%2CBriefTitle%2CCondition%2COverallStatus%2CLeadSponsorName%2CEligibilityCriteria&min_rnk=1&max_rnk=1000&fmt=csv


interactive(children=(Dropdown(description='column', options=('OverallStatus', 'Condition'), value='OverallSta…

## NLP
Now that I have data for a patient consolidated I can start to parse it! There's a few different pretrained parser models that I can try fairly easily. I'll be making use of the spacy, medspacy, and scispacy libraries to aid with this. 

In [13]:
#Load the models
ss_sm            = spacy.load("en_core_sci_sm")
#ss_md            = spacy.load("en_core_sci_md")
#ss_bert          = spacy.load("en_core_sci_scibert")
#ss_lg            = spacy.load("en_core_sci_lg")
#ss_craft         = spacy.load("en_ner_craft_md")
#ss_jnlpba        = spacy.load("en_ner_jnlpba_md")
#ss_bionlp13cg_md = spacy.load("en_ner_bionlp13cg_md")
#med7             = spacy.load("en_core_med7_lg")

In [20]:
#Check the components of the pipeline
ss_sm.pipe_names

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner']

Hmmm, there's no removal of stopwords here. I'll have to incorporate that. 

In [45]:
ss_sm.pipe_labels['ner']

['ENTITY']

In [17]:
# create distinct colours for labels
#col_dict = {}
#seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
#for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
#    col_dict[label] = colour

#options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}

#text = 'A patient was prescribed Magnesium hydroxide 400mg/5ml suspension PO of total 30ml bid for the next 5 days.'
#doc = med7(text)

#spacy.displacy.render(doc, style='ent', jupyter=True, options=options)

#[(ent.text, ent.label_) for ent in doc.ents]

In [16]:
#Select model to use in pipeline
#nlp = spacy.load("en_core_sci_sm")

#Add the abbreviation detector to the pipeline
#nlp.add_pipe("abbreviation_detector")

##Add the entity linker to the pipeline
#nlp.add_pipe("scispacy_linker", config={"linker_name": "umls","max_entities_per_mention": 6})

#Add negation to the pipeline
#nlp.add_pipe("negex")

In [19]:
#Define function to remove stopwords to be incorporated into the pipeline
@Language.component("remove_stopwords")
def remove_stopwords(doc):
  doc = [token for token in doc if not token.is_stop]
  print(doc)
  return doc

In [3]:
#Add a tokenizer at the beginning of the 
#ss_sm.add_pipe("tokenizer",first=True)

# Add the custom pipeline component to the pipeline after the tokenizer
#ss_sm.add_pipe('remove_stopwords', name='stopwords', after='lemmatizer')

#Check the components of the pipeline to ensure that the stopword removal is now a part of it
ss_sm.pipe_names

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner']

In [32]:
# Define regex pattern to match parentheses and punctuation
pattern = r"[\(\)\[\]\{\}\.,:;\!\?]"

# Create custom tokenizer with regex pattern to remove parentheses and punctuation
class CustomTokenizer:
    def __init__(self, ss_sm):
        self.tokenizer = ss_sm.tokenizer
        self.regex_pattern = re.compile(pattern)

    def __call__(self, text):
        tokens = self.tokenizer(text)
        new_tokens = []
        for token in tokens:
            # Remove tokens that match the regex pattern
            if self.regex_pattern.match(token.text) is None:
                new_tokens.append(token)
        return spacy.tokens.Doc(ss_sm.vocab, new_tokens)

# Add custom tokenizer to pipeline
ss_sm.tokenizer = CustomTokenizer(ss_sm)

In [5]:
text = "Myeloid derived suppressor cells (MDSC) are immature myeloid cells with immunosuppressive activity. \
They accumulate in tumor-bearing mice and humans with different types of cancer, including hepatocellular carcinoma (HCC)."
doc = ss_sm(text)

fmt_str = "{:<15}| {:<6}| {:<7}| {:<8}"
print(fmt_str.format("token", "pos", "label", "parent"))

for token in doc:
    print(fmt_str.format(token.text, token.pos_, token.ent_type_, token.head.text))

token          | pos   | label  | parent  
Myeloid        | ADJ   | ENTITY | derived 
derived        | ADJ   | ENTITY | cells   
suppressor     | NOUN  | ENTITY | cells   
cells          | NOUN  | ENTITY | cells   
(              | PUNCT |        | MDSC    
MDSC           | NOUN  | ENTITY | cells   
)              | PUNCT |        | MDSC    
are            | AUX   |        | cells   
immature       | ADJ   | ENTITY | cells   
myeloid        | ADJ   | ENTITY | cells   
cells          | NOUN  | ENTITY | cells   
with           | ADP   |        | activity
immunosuppressive| ADJ   | ENTITY | activity
activity       | NOUN  | ENTITY | cells   
.              | PUNCT |        | cells   
They           | PRON  |        | accumulate
accumulate     | VERB  | ENTITY | accumulate
in             | ADP   |        | mice    
tumor-bearing  | ADJ   | ENTITY | mice    
mice           | NOUN  | ENTITY | accumulate
and            | CCONJ |        | mice    
humans         | NOUN  | ENTITY | mice    
wit

In [34]:
#Text to be processed
text = "Myeloid derived suppressor cells (MDSC) are immature myeloid cells with immunosuppressive activity. \
They accumulate in tumor-bearing mice and humans with different types of cancer, including hepatocellular carcinoma (HCC)."

#Add the abbreviation pipe to the spacy pipeline.
if 'abbreviation_detector' not in ss_sm.pipe_names:
   ss_sm.add_pipe("abbreviation_detector")

#Add the EntityLinker pipe to spacy pipeline
if 'scispacy_linker' not in ss_sm.pipe_names:
  ss_sm.add_pipe("scispacy_linker", config={"linker_name": "umls", "max_entities_per_mention": 3})

#Create spacy doc from text
doc = ss_sm(text)

# Create a list of stopwords
stopwords = spacy.lang.en.stop_words.STOP_WORDS

#Remove stopwords and punctuations
filtered_tokens = [token.text      for token in doc if not token.is_stop and not token.is_punct]
filtered_pos    = [token.pos_      for token in doc if not token.is_stop and not token.is_punct]
filtered_label  = [token.ent_type_ for token in doc if not token.is_stop and not token.is_punct]
filtered_parent = [token.head.text for token in doc if not token.is_stop and not token.is_punct]
print('Parsing Text')

#PART 1. Basic NER and removal of punctuations/stopwords
fmt_str = "{:<20}| {:<6}| {:<7}| {:<8}"
print(fmt_str.format("token", "pos", "label", "parent"))

# Print filtered tokens and named entities after removal of punctuations and stopwords
for i in range(len(filtered_tokens)):
    print(fmt_str.format(filtered_tokens[i], filtered_pos[i], filtered_label[i], filtered_parent[i]))

#PART 2. Identification of abbreviations
print('\nAbbreviations')
fmt_str = "{:<6}| {:<35}| {:<6}| {:<6}"
print(fmt_str.format("Short", "Long", "Starts", "Ends"))

for abrv in doc._.abbreviations:
    print(fmt_str.format(abrv.text, str(abrv._.long_form), abrv.start, abrv.end))

#PART 3. Linking to medical concepts via UMLS
print('\nEntity Linking')
fmt_str = "{:<35}| {:<11}| {:<6}"
print(fmt_str.format("Entity", "Concept ID", "Score"))

for i in range(len(doc.ents)): #Print the entities and their associated UMLS codes
  entity = doc.ents[i]
  for kb_entry in entity._.kb_ents:
    cui = kb_entry[0]
    match_score = kb_entry[1]
    print(fmt_str.format(entity.text, cui, match_score))

Parsing Text
token               | pos   | label  | parent  
Myeloid             | ADJ   | ENTITY | derived 
derived             | ADJ   | ENTITY | cells   
suppressor          | NOUN  | ENTITY | cells   
cells               | NOUN  | ENTITY | cells   
MDSC                | NOUN  | ENTITY | cells   
immature            | ADJ   | ENTITY | cells   
myeloid             | ADJ   | ENTITY | cells   
cells               | NOUN  | ENTITY | cells   
immunosuppressive   | ADJ   | ENTITY | activity
activity            | NOUN  | ENTITY | cells   
accumulate          | VERB  | ENTITY | accumulate
tumor-bearing       | ADJ   | ENTITY | mice    
mice                | NOUN  | ENTITY | accumulate
humans              | NOUN  | ENTITY | mice    
different           | ADJ   |        | types   
types               | NOUN  |        | accumulate
cancer              | NOUN  | ENTITY | types   
including           | VERB  |        | carcinoma
hepatocellular      | ADJ   | ENTITY | carcinoma
carcinoma          

In [39]:
ss_sm.pipe_names

['tok2vec',
 'tagger',
 'attribute_ruler',
 'lemmatizer',
 'parser',
 'ner',
 'abbreviation_detector',
 'scispacy_linker']

## Parse Eligibility Criteria of Clinical Trials
For this I'll first focus on the EligibilityCriteria column on the qtrials dataframe.

In [48]:
qt_ec = qtrials[['EligibilityCriteria']]
qt_ec

,EligibilityCriteria
0,Inclusion Criteria:||Penicillin allergy tested...
1,"Inclusion Criteria:||All children, young peopl..."
2,Inclusion Criteria:||Pediatric Hospital Medici...
3,Inclusion Criteria:||Parents of children ages ...
4,Inclusion Criteria:||Adult patients who are re...
...,...
995,Inclusion Criteria:||Male and female subjects ...
996,Inclusion Criteria:||Male/female|Age ≥18 but <...
997,Inclusion Criteria:||exposure to mold at home|...
998,Key Inclusion Criteria:||Age 4 through 17 year...


In [71]:
ec = qt_ec['EligibilityCriteria'][1]
ec

'Inclusion Criteria:||All children, young people and adults who fit the selection criteria from across all the practices can be referred to the allergy clinic.||All patients and parents / carers where appropriate must be deemed capable of giving informed consent to take part in the research project.||Infants under two with suspected food allergy|Infants under two with moderate-to-severe eczema not responding to standard treatment.|Children and young people (up to 16 years of age) with suspected allergic rhinitis symptoms that are unresponsive to a combination of oral antihistamines and nasal steroids|Young people and adults (from 16 years of age) with a history of anaphylaxis or suspected anaphylaxis||Exclusion Criteria:||Over 2 years of age with delayed type food allergy presenting primarily with gastrointestinal symptoms|Over 2 years of age with confirmed non IgE-mediated symptoms including food intolerances, coeliac disease etc.|Single urticarial reactions without an obvious trigger

Each row contains the eligibility criteria for a clinical trials. The eligibility criteria is based on inclusion criteria and exclusion criteria. I think I'll start by splitting the eligibility criteria into two separate dataframes: 1 for the inclusion criteria and 1 for the exclusion criteria. This can be done fairly easily since the Inclusion and Exclusion Criteria segments are clearly demarcated.

In [58]:
sections = ec.split("Exclusion Criteria:||")
inclusion_criteria = sections[0].replace('Inclusion Criteria:||',"")
exclusion_criteria = sections[1]
print("Inclusion Criteria: ", inclusion_criteria)
print("Exclusion Criteria: ", exclusion_criteria)

Inclusion Criteria:  All children, young people and adults who fit the selection criteria from across all the practices can be referred to the allergy clinic.||All patients and parents / carers where appropriate must be deemed capable of giving informed consent to take part in the research project.||Infants under two with suspected food allergy|Infants under two with moderate-to-severe eczema not responding to standard treatment.|Children and young people (up to 16 years of age) with suspected allergic rhinitis symptoms that are unresponsive to a combination of oral antihistamines and nasal steroids|Young people and adults (from 16 years of age) with a history of anaphylaxis or suspected anaphylaxis||
Exclusion Criteria:  Over 2 years of age with delayed type food allergy presenting primarily with gastrointestinal symptoms|Over 2 years of age with confirmed non IgE-mediated symptoms including food intolerances, coeliac disease etc.|Single urticarial reactions without an obvious trigger

In [66]:
print('Inclusion Criteria')
ic_list,ec_list = [],[]
for item in  inclusion_criteria.split('||'):
    ic_list.extend(item.split("|"))
print('\n'.join(ic_list))

print('Exclusion Criteria')
for item in  exclusion_criteria.split('||'):
    ec_list.extend(item.split("|"))
print('\n'.join(ec_list))

Inclusion Criteria
All children, young people and adults who fit the selection criteria from across all the practices can be referred to the allergy clinic.
All patients and parents / carers where appropriate must be deemed capable of giving informed consent to take part in the research project.
Infants under two with suspected food allergy
Infants under two with moderate-to-severe eczema not responding to standard treatment.
Children and young people (up to 16 years of age) with suspected allergic rhinitis symptoms that are unresponsive to a combination of oral antihistamines and nasal steroids
Young people and adults (from 16 years of age) with a history of anaphylaxis or suspected anaphylaxis

Exclusion Criteria
Over 2 years of age with delayed type food allergy presenting primarily with gastrointestinal symptoms
Over 2 years of age with confirmed non IgE-mediated symptoms including food intolerances, coeliac disease etc.
Single urticarial reactions without an obvious triggers
Non-a

Nice! Now that I have these criteria separated, I'll place them into dataframes

In [69]:
#Dataframe containing inclusion criteria for current trial
ic_df = pd.DataFrame(ic_list,columns=['InclusionCriteria'])
ic_df

,InclusionCriteria
0,"All children, young people and adults who fit ..."
1,All patients and parents / carers where approp...
2,Infants under two with suspected food allergy
3,Infants under two with moderate-to-severe ecze...
4,Children and young people (up to 16 years of a...
5,Young people and adults (from 16 years of age)...
6,


In [70]:
#Dataframe containing exclusion criteria for current trial
ec_df = pd.DataFrame(ec_list,columns=['ExclusionCriteria'])
ec_df

,ExclusionCriteria
0,Over 2 years of age with delayed type food all...
1,Over 2 years of age with confirmed non IgE-med...
2,Single urticarial reactions without an obvious...
3,Non-allergic chronic urticaria
4,Drug allergy
5,"Well controlled allergic rhinitis, asthma or a..."
6,Mild-to-moderate atopic eczema without an obvi...
7,Localised insect sting reactions


In [75]:
#Tokenize and remove stopwords and punctuation from each row 
ic_df['tokens'] = ic_df['InclusionCriteria'].apply(lambda x: [token.text      for token in ss_sm(x)  if not token.is_stop and not token.is_punct])
ic_df['pos']    = ic_df['InclusionCriteria'].apply(lambda x: [token.pos_      for token in ss_sm(x)  if not token.is_stop and not token.is_punct])
ic_df['entity'] = ic_df['InclusionCriteria'].apply(lambda x: [token.ent_type_ for token in ss_sm(x)  if not token.is_stop and not token.is_punct])
ic_df['parent'] = ic_df['InclusionCriteria'].apply(lambda x: [token.head.text for token in ss_sm(x)  if not token.is_stop and not token.is_punct])
ic_df

,InclusionCriteria,tokens,pos,entity,parent
0,"All children, young people and adults who fit ...","[children, young, people, adults, fit, selecti...","[NOUN, ADJ, NOUN, NOUN, VERB, NOUN, NOUN, NOUN...","[ENTITY, ENTITY, ENTITY, ENTITY, , ENTITY, ENT...","[referred, people, referred, people, adults, c..."
1,All patients and parents / carers where approp...,"[patients, parents, carers, appropriate, deeme...","[NOUN, NOUN, NOUN, NOUN, VERB, ADJ, VERB, ADJ,...","[ENTITY, ENTITY, ENTITY, , , , , , ENTITY, ENT...","[patients, patients, patients, where, carers, ..."
2,Infants under two with suspected food allergy,"[Infants, suspected, food, allergy]","[NOUN, VERB, NOUN, NOUN]","[ENTITY, ENTITY, ENTITY, ENTITY]","[Infants, allergy, allergy, Infants]"
3,Infants under two with moderate-to-severe ecze...,"[Infants, moderate-to-severe, eczema, respondi...","[NOUN, ADJ, NOUN, VERB, ADJ, NOUN]","[ENTITY, , ENTITY, , ENTITY, ENTITY]","[Infants, eczema, two, Infants, treatment, res..."
4,Children and young people (up to 16 years of a...,"[Children, young, people, 16, years, age, susp...","[NOUN, ADJ, NOUN, NUM, NOUN, NOUN, ADJ, ADJ, N...","[ENTITY, ENTITY, ENTITY, , ENTITY, ENTITY, , E...","[Children, people, Children, years, Children, ..."
5,Young people and adults (from 16 years of age)...,"[Young, people, adults, 16, years, age, histor...","[ADJ, NOUN, NOUN, NUM, NOUN, NOUN, NOUN, ADJ, ...","[ENTITY, ENTITY, ENTITY, , ENTITY, ENTITY, ENT...","[people, people, people, years, people, years,..."
6,,[],[],[],[]
